In [208]:
import re
import numpy as np
import pandas as pd

In [209]:
filename = 'airbnb-listings-FR'
data = pd.read_csv(filename+'.csv', delimiter=';', decimal='.', low_memory=False)
COUNTRY_CODE = data['Country Code'][0]
COUNTRY_CODE

'FR'

In [210]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56562 entries, 0 to 56561
Data columns (total 89 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              56562 non-null  int64  
 1   Listing Url                     56562 non-null  object 
 2   Scrape ID                       56562 non-null  int64  
 3   Last Scraped                    56562 non-null  object 
 4   Name                            56559 non-null  object 
 5   Summary                         54397 non-null  object 
 6   Space                           39155 non-null  object 
 7   Description                     56555 non-null  object 
 8   Experiences Offered             56562 non-null  object 
 9   Neighborhood Overview           33785 non-null  object 
 10  Notes                           18012 non-null  object 
 11  Transit                         36237 non-null  object 
 12  Access                          

In [211]:
cols_to_drop = []

# Pick out columns with missing values more than half the size of the dataset
for col in data.columns:
    if data[col].isnull().sum() > len(data)/2:
        cols_to_drop.append(col)
        
# Drop all the columns that start with 'Host'
for col in data.columns:
    if col.startswith('Host'): cols_to_drop.append(col)
        
for x in ['Scrape ID', 'Last Scraped', 'Summary', 'Space', 'Description', 'Neighbourhood', 'Neighbourhood Group Cleansed', 'Neighborhood Overview']: cols_to_drop.append(x)
for x in ['Name', 'Listing Url', 'Transit', 'Access', 'Interaction', 'House Rules', 'Thumbnail Url', 'Medium Url', 'XL Picture Url']: cols_to_drop.append(x)
for x in ['Street', 'State', 'Zipcode', 'Market', 'Smart Location', 'Latitude', 'Longitude', 'Country', 'Beds', 'Bed Type']: cols_to_drop.append(x)
for x in ['Experiences Offered', 'Cleaning Fee', 'Guests Included', 'Extra People', 'Minimum Nights', 'Maximum Nights']: cols_to_drop.append(x)
for x in ['Calendar Updated', 'Calendar last Scraped', 'Number of Reviews', 'First Review', 'Last Review', 'Reviews per Month']: cols_to_drop.append(x)
for x in ['Review Scores Checkin', 'Jurisdiction Names', 'Cancellation Policy', 'Calculated host listings count']: cols_to_drop.append(x)
for x in ['Geolocation', 'Features']: cols_to_drop.append(x)
print(cols_to_drop, len(cols_to_drop))

['Notes', 'Interaction', 'Picture Url', 'Host Acceptance Rate', 'Neighbourhood Group Cleansed', 'Square Feet', 'Weekly Price', 'Monthly Price', 'Has Availability', 'License', 'Host ID', 'Host URL', 'Host Name', 'Host Since', 'Host Location', 'Host About', 'Host Response Time', 'Host Response Rate', 'Host Acceptance Rate', 'Host Thumbnail Url', 'Host Picture Url', 'Host Neighbourhood', 'Host Listings Count', 'Host Total Listings Count', 'Host Verifications', 'Scrape ID', 'Last Scraped', 'Summary', 'Space', 'Description', 'Neighbourhood', 'Neighbourhood Group Cleansed', 'Neighborhood Overview', 'Name', 'Listing Url', 'Transit', 'Access', 'Interaction', 'House Rules', 'Thumbnail Url', 'Medium Url', 'XL Picture Url', 'Street', 'State', 'Zipcode', 'Market', 'Smart Location', 'Latitude', 'Longitude', 'Country', 'Beds', 'Bed Type', 'Experiences Offered', 'Cleaning Fee', 'Guests Included', 'Extra People', 'Minimum Nights', 'Maximum Nights', 'Calendar Updated', 'Calendar last Scraped', 'Number 

In [212]:
data = data.drop(cols_to_drop, axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56562 entries, 0 to 56561
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           56562 non-null  int64  
 1   Neighbourhood Cleansed       56562 non-null  object 
 2   City                         56532 non-null  object 
 3   Country Code                 56562 non-null  object 
 4   Property Type                56562 non-null  object 
 5   Room Type                    56562 non-null  object 
 6   Accommodates                 56562 non-null  int64  
 7   Bathrooms                    56378 non-null  float64
 8   Bedrooms                     56428 non-null  float64
 9   Amenities                    56134 non-null  object 
 10  Price                        56505 non-null  float64
 11  Security Deposit             31127 non-null  float64
 12  Availability 30              56562 non-null  int64  
 13  Availability 60 

In [213]:
data = data.dropna()

In [214]:
# Remove 'translation missing:' tags in the amentity column of the dataset
data['Amenities'] = data['Amenities'].str.replace(',translation missing: en.hosting_amenity_49', '')
data['Amenities'] = data['Amenities'].str.replace('translation missing: en.hosting_amenity_49,', '')
data['Amenities'] = data['Amenities'].str.replace(',translation missing: en.hosting_amenity_50', '')
data['Amenities'] = data['Amenities'].str.replace('translation missing: en.hosting_amenity_50,', '')

In [215]:
# If the neighbourhood cleansed column has a integer value, drop the record
def convert_num_nan(x):
    if x.isnumeric(): return np.nan
    return x

data['Neighbourhood Cleansed'] = data['Neighbourhood Cleansed'].apply(convert_num_nan)
data = data.dropna()

In [216]:
# Normalise city names (avoid duplicated names)
def normalise_cities(x: str):
    x = x.lower()
    if "," in x: x = x.split(",")[1].strip()
    pattern = re.compile(r'[^\u0000-\u007F0-9\s]')
    x = pattern.sub('', x)
    if x == COUNTRY_CODE.lower(): return None
    if x == '' or x.isspace(): return None
    if COUNTRY_CODE == 'FR' and 'paris' in x: return 'Paris'
    return " ".join(word.strip().capitalize() for word in x.split(' '))

data['City'] = data['City'].apply(normalise_cities)
data = data.dropna()

In [217]:
data['City'].unique()

array(['Paris', 'Montrouge', 'Boulogne-billancourt', 'Saint-mand',
       'Charenton-le-pont', 'Neuilly-sur-seine', 'Levallois-perret',
       'Le Pr-saint-gervais', 'Issy-les-moulineaux', 'Vincennes',
       'Aubervilliers', 'Issy Les Moulineaux', 'Le Kremlin-bictre',
       'Le-de-france', 'Nogent-sur-marne', 'Gaillard',
       'Rer D  Une Statio Du Chatelt', 'Malakoff', 'Deuil-la-barre',
       'Parigi', 'Saint-ouen', 'Saint-julien-en-genevois',
       'Collonges-sous-salve', 'Fontenay-sous-bois', 'Viry', 'Clichy',
       'Pars', 'Vanves', 'Pris', 'Les Lilas', 'Gentilly', 'Pantin',
       'Parijs', 'Neuilly Sur Seine', 'Joinville-le-pont',
       'Saint-maurice', 'Ivry-sur-seine', 'Montreuil', 'Saint-denis'],
      dtype=object)

In [218]:
if len(data) > 50000: data = data.sample(50000)
data.to_csv(filename + '-clean.csv', index=False)

In [219]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24653 entries, 0 to 56561
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           24653 non-null  int64  
 1   Neighbourhood Cleansed       24653 non-null  object 
 2   City                         24653 non-null  object 
 3   Country Code                 24653 non-null  object 
 4   Property Type                24653 non-null  object 
 5   Room Type                    24653 non-null  object 
 6   Accommodates                 24653 non-null  int64  
 7   Bathrooms                    24653 non-null  float64
 8   Bedrooms                     24653 non-null  float64
 9   Amenities                    24653 non-null  object 
 10  Price                        24653 non-null  float64
 11  Security Deposit             24653 non-null  float64
 12  Availability 30              24653 non-null  int64  
 13  Availability 60      